In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import numpy as np
import random
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, accuracy_score
import os
from sklearn.utils import shuffle
import time
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers

def cotan(angle):
    return -np.tan(angle + np.pi/2)

# Function to convert ellipse to rectangular box
def ellipse_to_rectangle(major_axis, minor_axis, angle, center_x, center_y):
    t1 = np.arctan(-minor_axis * np.tan(angle) / major_axis)
    t2 = np.arctan(minor_axis * cotan(angle) / major_axis)
    xoff = abs(major_axis * np.cos(t1) * np.cos(angle) - minor_axis * np.sin(t1) * np.sin(angle))
    yoff = abs(minor_axis * np.sin(t2) * np.cos(angle) - major_axis * np.cos(t2) * np.sin(angle))
    x1 = center_x - xoff
    y1 = center_y - yoff
    x2 = center_x + xoff
    y2 = center_y + yoff
    return [x1, y1, x2, y2]

def get_image(image_path, scale=6):
    image = cv2.imread(image_path)
    new_height = int(image.shape[0] / scale)
    new_width = int(image.shape[1] / scale)
    return cv2.resize(image, (new_width, new_height))

# Parse FDDB dataset files and convert ellipse annotations to rectangles.
# Split into training and testing datasets.
def parse_fddb_dataset(folder_path, num_training=8):
    training_data = []
    testing_data = []

    for i in range(1, 11):
        file_path = os.path.join(folder_path, f'FDDB-fold-{i:02}-ellipseList.txt')
        with open(file_path, 'r') as file:
            lines = file.readlines()

        current_image = None
        face_count = 0
        get_face_count = False
        for line in lines:
            if get_face_count:
                face_count = int(line.strip())
                get_face_count = False
            elif face_count == 0:
                if current_image is not None:
                    # Process previous image
                    if i <= num_training:
                        training_data.append((current_image, faces))
                    else:
                        testing_data.append((current_image, faces))

                current_image = os.path.join('/content/drive/MyDrive/Colab Notebooks/CV_dataset/originalPics/', line.strip() + '.jpg')
                faces = []
                get_face_count = True
            else:
                parts = [float(part) for part in line.split()]
                rect = ellipse_to_rectangle(*parts[:5])
                faces.append(rect)
                face_count -= 1

        # Process the last image
        if current_image is not None:
            if i <= num_training:
                training_data.append((current_image, faces))
            else:
                testing_data.append((current_image, faces))

    return training_data, testing_data

def calculate_iou(boxA, boxB):
    # Determine the coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # Compute the area of intersection
    intersection_area = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # Compute the area of both bounding boxes
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # Compute the intersection over union
    iou = intersection_area / float(boxAArea + boxBArea - intersection_area)
    return iou

def sliding_window(image, step_size, window_size):
    for x in range(0, image.shape[1], step_size):
        for y in range(0, image.shape[0], step_size):
            yield (x, y, image[y:y + window_size[1], x:x + window_size[0]])

def generate_samples_using_sliding_window(image, faces, window_size, step_size, iou_threshold, scale=6):
    positive_samples = []
    negative_samples = []

    for (x, y, window) in sliding_window(image, step_size, window_size):
        # if window.shape[0] != window_size[1] or window.shape[1] != window_size[0]:
        #     continue

        scaled_box = [x * scale, y * scale, (x + window_size[0]) * scale, (y + window_size[1]) * scale]
        # print(scaled_box, faces)
        ious = [calculate_iou(scaled_box, face) for face in faces]
        max_iou = max(ious) if ious else 0
        # print(max_iou)

        current_box = [x, y, x + window_size[0], y + window_size[1]]
        if max_iou >= iou_threshold:
            positive_samples.append(current_box)
        else:
            negative_samples.append(current_box)

    return positive_samples, negative_samples

# Function to extract features in an image
def extract_features(image, box):
    # don't include this image if a dimension = 0
    # if int(box[3]) == int(box[1]) or int(box[2]) == int(box[0]):
    #     return [-1]
    cropped_image = image[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
    # feature_vector = cropped_image.flatten()
    # feature_vector = cv2.resize(cropped_image, (16, 16)).flatten()
    feature_vector = cv2.resize(cropped_image, (32, 32))
    return feature_vector

def extract_dataset_features(data):
    # Prepare data for classifier
    # X, y = [], []
    X_pos, X_neg, y_pos, y_neg = [], [], [], []
    count_pos = 0
    count_neg = 0
    scale = 3
    for image_path, boxes in data:
        image = get_image(image_path, scale)
        pos_samples, neg_samples = generate_samples_using_sliding_window(image, boxes, window_size=(32, 32), step_size=4, iou_threshold=0.5, scale=scale)
        # print(neg_samples)
        for box in pos_samples:
            feature = extract_features(image, box)
            # if feature[0] == -1:
            #     continue
            count_pos += 1
            X_pos.append(feature)
            y_pos.append(1)  # Label for face

        for neg_box in neg_samples:
            feature = extract_features(image, neg_box)
            # if feature[0] == -1:
            #     continue
            count_neg += 1
            X_neg.append(feature)
            y_neg.append(0)  # Label for non-face

    # standardize the dataset
    if count_pos < count_neg:
        data_size = count_pos
        random_rows = random.sample(range(count_neg), data_size)
        X_neg = [X_neg[i] for i in random_rows]
        # y_neg = [y_neg[i] for i in random_rows]
        y_neg = [0] * data_size
    else:
        data_size = count_neg
        random_rows = random.sample(range(count_pos), data_size)
        X_pos = [X_pos[i] for i in random_rows]
        # y_pos = [y_pos[i] for i in random_rows]
        y_pos = [1] * data_size

    # y_pos_np = np.array(data_size * [[0,1]])
    # y_neg_np = np.array(data_size * [[1,0]])

    X = np.array(X_pos + X_neg)
    y = np.array(y_pos + y_neg)
    # y = np.concatenate((y_pos_np, y_neg_np))

    X, y = shuffle(X, y)

    return X, y

training_data, testing_data = parse_fddb_dataset('/content/drive/MyDrive/Colab Notebooks/CV_dataset/FDDB-folds')

X_train, y_train = extract_dataset_features(training_data)
X_test, y_test = extract_dataset_features(testing_data)

In [23]:
def generate_random_haar_features(num_features):
    features = []
    for i in range(num_features):
        # (x, y) is the position, w = width, h = height
        w = np.random.randint(0, 32)
        h = np.random.randint(0, 32)
        x = np.random.randint(0, 32 - w)
        y = np.random.randint(0, 32 - h)
        features.append((w, h, x, y))
    return np.array(features)

def compute_integral_image(image):
    # grayscale image
    grayscale = cv2.cvtColor(image.reshape(32, 32, 3), cv2.COLOR_RGB2GRAY)

    integral_image = cv2.integral(grayscale)

    return integral_image

def apply_haar_features(integral_image, features):
    feature_values = []
    for feature in features:
        w, h, x, y = feature
        white_rectangle = integral_image[y + h, x + w // 2] - integral_image[y, x + w // 2] - integral_image[y + h, x] + integral_image[y, x]
        black_rectangle = integral_image[y + h, x + w] - integral_image[y + h, x + w // 2] - integral_image[y, x + w] + integral_image[y, x + w // 2]
        feature_values.append(white_rectangle - black_rectangle)

    return np.array(feature_values)

In [21]:
# train Adaboost classifiers for each class (one-versus-all)
# this uses the weak feature computation (haar features)
def adaboost_train(X_train, y_train, num_features, class_label, T):
    y_train_binary = (y_train == class_label)

    weak_classifiers = []

    # initialize weights based on y_i = 0 or 1
    weights = np.where(y_train == 0, 1.0 / (2 * np.sum(y_train_binary == 0)), 1.0 / (2 * np.sum(y_train_binary == 1)))

    # for t = 1...T
    for t in range(T):
        # normalize the weights
        weights = weights / np.sum(weights)

        # train a classifier for each feature
        min_error = float('inf')
        best_feature_index = 1
        best_threshold = 1
        for j in range(num_features):
            features = X_train[:, j]
            threshold = np.random.uniform(min(features), max(features))
            predictions = np.where(features >= threshold, 1, 0)

            error = np.sum(weights * abs(predictions - y_train_binary))

            if error < min_error:
                min_error = error
                best_feature_index = j
                # choose best classifier with lowest error
                best_threshold = threshold

        if min_error == float('inf'):
            continue

        if min_error == 0:
            break

        beta = (1 - min_error) / min_error
        alpha = np.log(1 / beta)

        # update the weights
        predictions = np.where(X_train[:, best_feature_index] >= best_threshold, 1, 0)
        weights *= np.power(beta, 1 - abs(predictions - y_train_binary))

        # add to full array of classifiers
        weak_classifiers.append([alpha, best_threshold, best_feature_index])

    return weak_classifiers

In [ ]:
# multi-class image classifier combining one-versus-all classifiers
def train_multiclass_adaboost(X_train, y_train, T):
    num_samples, num_features = X_train.shape

    classes = np.unique(y_train)
    num_classes = len(classes)

    classifiers_list = []

    for i in range(num_classes):
        # Train one-versus-all classifier for each class
        classifiers_list.append(adaboost_train(X_train, y_train, num_features, classes[i], T))

    return classifiers_list

In [39]:
def adaboost_predict(X_test, classifiers_list):
    num_samples, num_features = X_test.shape

    predictions = np.zeros((num_samples, len(classifiers_list)))

    for i, classifier in enumerate(classifiers_list):
        current_scores = np.zeros(num_samples)
        # calculate scores for each classifier in classifier list
        for alpha, threshold, feature_index in classifier:
            weak_predictions = X_test[:, feature_index] >= threshold
            current_scores += alpha * weak_predictions
        predictions[:, i] = current_scores

    # return the class with the highest score
    return np.argmax(predictions, axis=1)

In [26]:
features = generate_random_haar_features(1000)
X_train_integral_images = [compute_integral_image(image) for image in X_train]
X_test_integral_images = [compute_integral_image(image) for image in X_test]

X_train_haar = np.array([apply_haar_features(image, features) for image in X_train_integral_images])
X_test_haar = np.array([apply_haar_features(image, features) for image in X_test_integral_images])

In [ ]:
start_time = time.time()
adaboost_classifier_list = train_multiclass_adaboost(X_train_haar, y_train, 1)
training_time = time.time() - start_time
print(f"Training completed in {training_time:.2f} seconds.")

# Calculate accuracy
predictions = adaboost_predict(X_test_haar, adaboost_classifier_list)
predictions = [0 if x == 1 else 1 for x in predictions]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

precision, recall, _ = precision_recall_curve(y_test, predictions)

# Plotting the Precision-Recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.', label='Adaboost')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve')
plt.legend()
plt.show()